Anticipez les besoins en consommation électrique de bâtiments
=============================================================

![logo-seattle](../reports/figures/logo-seattle.png)


Explication des variables:
[City of seattle](https://data.seattle.gov/dataset/2015-Building-Energy-Benchmarking/h7rm-fz6m)

In [ ]:
import numpy as np
import pandas as pd

import plotly.express as px
import seaborn as sns
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import decomposition

from src.utils.bivar import BivariateAnalysis

sns.set()

In [ ]:
data = pd.read_pickle('../data/interim/full_data.pickle')

In [ ]:
data.columns

Variable à prédire (targets)

   * SiteEnergyUse/WN (wether normalized?)
   * TotalGHGEmissions

Variables sélectionnées comme entrée du modèle
    
   * Surface du bâtiment
   * Premiere utilisation du bâtiment
   * Surface destinee au premier usage du batiment
    

## Préparation des variables pour modélisation

suppression des lignes dont la consomation est nulle

In [ ]:
data = data[data["SiteEnergyUseWN_kBtu"] > 0]

On supprime également les lignes pour lesquelles on a pas l'usage principal du bâtiment

In [ ]:
data = data[data['LargestPropertyUseType'].notna()]

## Quelles sont les variables corrélées avec la cible?

In [ ]:
bivar = BivariateAnalysis(data)
bivar.anova('SiteEnergyUseWN_kBtu', 'LargestPropertyUseType', orient='h')

In [ ]:
bivar.regression(['PropertyGFATotal', 'SiteEnergyUseWN_kBtu'])

**Notes** : Passage en log pour les colonnes PropertyGFATotal et SiteEnergyUseWN_kBtu!

In [ ]:
data['logPropertyGFATotal'] = data['PropertyGFATotal'].apply(np.log1p)
data['logSiteEnergyUseWN_kBtu'] = data['SiteEnergyUseWN_kBtu'].apply(np.log1p)

In [ ]:
bivar = BivariateAnalysis(data)
bivar.regression(['logPropertyGFATotal', 'logSiteEnergyUseWN_kBtu'])


Meilleure corrélation entre les logs des variables

In [ ]:
fig = px.scatter(data, x='logPropertyGFATotal', 
                 y='logSiteEnergyUseWN_kBtu',
                 hover_data=['PropertyName'],
                 color='LargestPropertyUseType')
fig.show()

In [ ]:
data.to_pickle('../data/interim/full_dataV2.pickle')